## Training your own openWakeWord models


In [1]:


# Imports
import sys
import numpy as np
import torch
import sys
from pathlib import Path
import uuid
import yaml
import datasets
import scipy
from tqdm import tqdm
import locale
import os
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

# install openwakeword (full installation to support training)
if not os.path.exists("./openwakeword"):
    !git clone https://github.com/dscripka/openwakeword
    !pip install -e ./openwakeword --no-deps


os.makedirs("./openwakeword/openwakeword/resources/models", exist_ok=True)
!wget https://github.com/dscripka/openWakeWord/releases/download/v0.5.1/embedding_model.onnx -O ./openwakeword/openwakeword/resources/models/embedding_model.onnx
!wget https://github.com/dscripka/openWakeWord/releases/download/v0.5.1/embedding_model.tflite -O ./openwakeword/openwakeword/resources/models/embedding_model.tflite
!wget https://github.com/dscripka/openWakeWord/releases/download/v0.5.1/melspectrogram.onnx -O ./openwakeword/openwakeword/resources/models/melspectrogram.onnx
!wget https://github.com/dscripka/openWakeWord/releases/download/v0.5.1/melspectrogram.tflite -O ./openwakeword/openwakeword/resources/models/melspectrogram.tflite



resources_dir = "./resources"
if not os.path.exists(resources_dir):
    os.mkdir(resources_dir)
## Download all data

## Download MIR RIR data (takes about ~2 minutes)
mit_rirs_dir ="mit_rirs"

mit_rirs_dir_path = os.path.join(resources_dir, mit_rirs_dir)



if not os.path.exists(mit_rirs_dir_path):
    os.mkdir(mit_rirs_dir_path)
    !git lfs install
    !git clone https://huggingface.co/datasets/davidscripka/MIT_environmental_impulse_responses
    !mv MIT_environmental_impulse_responses {resources_dir}/MIT_environmental_impulse_responses
    rir_dataset = datasets.Dataset.from_dict({"audio": [str(i) for i in Path(os.path.join(resources_dir, "MIT_environmental_impulse_responses","16khz")).glob("*.wav")]}).cast_column("audio", datasets.Audio())
    # Save clips to 16-bit PCM wav files
    for row in tqdm(rir_dataset):
        name = row['audio']['path'].split('/')[-1]
        scipy.io.wavfile.write(os.path.join(mit_rirs_dir_path, name), 16000, (row['audio']['array']*32767).astype(np.int16))

## Download noise and background audio (takes about ~3 minutes)

# Audioset Dataset (https://research.google.com/audioset/dataset/index.html)
# Download one part of the audioset .tar files, extract, and convert to 16khz
# For full-scale training, it's recommended to download the entire dataset from
# https://huggingface.co/datasets/agkphysics/AudioSet, and
# even potentially combine it with other background noise datasets (e.g., FSD50k, Freesound, etc.)

audioset_dir=os.path.join(resources_dir, "audioset")
if not os.path.exists(audioset_dir):
    os.mkdir(audioset_dir)

    fname = "bal_train09.tar"
    out_dir = os.path.join(audioset_dir, fname)
    link = "https://huggingface.co/datasets/agkphysics/AudioSet/resolve/main/data/" + fname
    !wget -O {out_dir} {link}
    !cd {audioset_dir}  && tar -xvf bal_train09.tar

    output_dir = os.path.join(resources_dir, "audioset_16k")
    if not os.path.exists(output_dir):
        os.mkdir(output_dir)

    # Save clips to 16-bit PCM wav files
    audioset_dataset = datasets.Dataset.from_dict({"audio": [str(i) for i in Path(audioset_dir, "audio").glob("**/*.flac")]})
    audioset_dataset = audioset_dataset.cast_column("audio", datasets.Audio(sampling_rate=16000))
    for row in tqdm(audioset_dataset):
        name = row['audio']['path'].split('/')[-1].replace(".flac", ".wav")
        scipy.io.wavfile.write(os.path.join(output_dir, name), 16000, (row['audio']['array']*32767).astype(np.int16))

# Free Music Archive dataset
# https://github.com/mdeff/fma

output_dir = os.path.join(resources_dir, "fma")
if not os.path.exists(output_dir):
    os.mkdir(output_dir)
    fma_dataset = datasets.load_dataset("rudraml/fma", name="small", split="train", streaming=True)
    fma_dataset = iter(fma_dataset.cast_column("audio", datasets.Audio(sampling_rate=16000)))

    # Save clips to 16-bit PCM wav files
    n_hours = 1  # use only 1 hour of clips for this example notebook, recommend increasing for full-scale training
    for i in tqdm(range(n_hours*3600//30)):  # this works because the FMA dataset is all 30 second clips
        row = next(fma_dataset)
        name = row['audio']['path'].split('/')[-1].replace(".mp3", ".wav")
        scipy.io.wavfile.write(os.path.join(output_dir, name), 16000, (row['audio']['array']*32767).astype(np.int16))
        i += 1
        if i == n_hours*3600//30:
            break

# Download pre-computed openWakeWord features for training and validation

# training set (~2,000 hours from the ACAV100M Dataset)
# See https://huggingface.co/datasets/davidscripka/openwakeword_features for more information
if not os.path.exists( os.path.join(resources_dir, "openwakeword_features_ACAV100M_2000_hrs_16bit.npy")):
    save_path = os.path.join(resources_dir, "openwakeword_features_ACAV100M_2000_hrs_16bit.npy")
    !wget https://huggingface.co/datasets/davidscripka/openwakeword_features/resolve/main/openwakeword_features_ACAV100M_2000_hrs_16bit.npy -O {save_path}

# validation set for false positive rate estimation (~11 hours)
if not os.path.exists(os.path.join(resources_dir, "validation_set_features.npy")):
    save_path = os.path.join(resources_dir, "validation_set_features.npy")
    !wget https://huggingface.co/datasets/davidscripka/openwakeword_features/resolve/main/validation_set_features.npy -O {save_path}


--2025-10-28 06:37:54--  https://github.com/dscripka/openWakeWord/releases/download/v0.5.1/embedding_model.onnx
Resolving github.com (github.com)... 140.82.114.3
Connecting to github.com (github.com)|140.82.114.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://release-assets.githubusercontent.com/github-production-release-asset/497407399/0233db07-b8db-4fc3-b026-b75d77fd7ae6?sp=r&sv=2018-11-09&sr=b&spr=https&se=2025-10-28T07%3A28%3A07Z&rscd=attachment%3B+filename%3Dembedding_model.onnx&rsct=application%2Foctet-stream&skoid=96c2d410-5711-43a1-aedd-ab1947aa7ab0&sktid=398a6654-997b-47e9-b12b-9515b896b4de&skt=2025-10-28T06%3A27%3A33Z&ske=2025-10-28T07%3A28%3A07Z&sks=b&skv=2018-11-09&sig=%2F8H5StiPm%2F%2FEiD%2BV3xz%2FRDkja0W4qUm%2BP0gM31MMbi0%3D&jwt=eyJ0eXAiOiJKV1QiLCJhbGciOiJIUzI1NiJ9.eyJpc3MiOiJnaXRodWIuY29tIiwiYXVkIjoicmVsZWFzZS1hc3NldHMuZ2l0aHVidXNlcmNvbnRlbnQuY29tIiwia2V5Ijoia2V5MSIsImV4cCI6MTc2MTYzMzc1MSwibmJmIjoxNzYxNjMzNDUxLCJwYXRoIjoicmVsZWFzZW

In [2]:
# Load default YAML config file for training
import yaml
from datetime import datetime

# 生成当前时间字符串，例如 2025-10-24_15-32-45
timestamp = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
target_word="hi_aldelo"
config = yaml.load(open("openwakeword/examples/custom_model.yml", 'r').read(), yaml.Loader)

# Modify values in the config and save a new version
number_of_training_steps = 20000  # @param {type:"slider", min:0, max:50000, step:100}
false_activation_penalty = 1500  # @param {type:"slider", min:100, max:5000, step:50}
config["target_phrase"] = [target_word]
config["model_name"] = f"{config['target_phrase'][0].replace(' ', '_')}_{timestamp}"
config["steps"] = number_of_training_steps
config["output_dir"] = "./trained_models/"
config["max_negative_weight"] = false_activation_penalty
config["datasets"]="./dataset"
config["background_paths"] = [os.path.join(resources_dir, "audioset_16k"), os.path.join(resources_dir, "fma")]  # multiple background datasets are supported
config["false_positive_validation_data_path"] = os.path.join(resources_dir, "validation_set_features.npy")
config["feature_data_files"] = {"ACAV100M_sample": os.path.join(resources_dir, "openwakeword_features_ACAV100M_2000_hrs_16bit.npy")}
config["rir_paths"]= [mit_rirs_dir_path]

config["n_samples"] = 10
config["n_samples_val"] = 10
config["negative_data"] = "negative_data"
config_yaml_path = "my_model.yaml"
with open(config_yaml_path, 'w') as file:
    documents = yaml.dump(config, file)







In [3]:
import torch
from torch import optim, nn
import torchinfo
import torchmetrics
import copy
import os
import sys
import tempfile
import uuid
import numpy as np
import scipy
import collections
import argparse
import logging
from tqdm import tqdm
import yaml
from pathlib import Path
from loguru import logger
from openwakeword.data import augment_clips, mmap_batch_generator
from openwakeword.utils import compute_features_from_generator

from openwakeword.train import Model
import shutil
# Separate function to convert onnx models to tflite format
def convert_onnx_to_tflite(onnx_model_path, output_path):
    """Converts an ONNX version of an openwakeword model to the Tensorflow tflite format."""
    # imports
    import onnx
    from onnx_tf.backend import prepare
    import tensorflow as tf

    # Convert to tflite from onnx model
    onnx_model = onnx.load(onnx_model_path)
    tf_rep = prepare(onnx_model, device="CPU")
    with tempfile.TemporaryDirectory() as tmp_dir:
        tf_rep.export_graph(os.path.join(tmp_dir, "tf_model"))
        converter = tf.lite.TFLiteConverter.from_saved_model(os.path.join(tmp_dir, "tf_model"))
        tflite_model = converter.convert()

        logging.info(f"####\nSaving tflite mode to '{output_path}'")
        with open(output_path, 'wb') as f:
            f.write(tflite_model)

    return None

parser = argparse.ArgumentParser()
parser.add_argument(
    "--training_config",
    help="The path to the training config file (required)",
    type=str,
    default=config_yaml_path,
)
parser.add_argument(
    "--generate_clips",
    help="Execute the synthetic data generation process",
    type=str,
    default="True",
)
parser.add_argument(    
    "--augment_clips",
    help="Execute the synthetic data augmentation process",
    type=str,
    default="True",
)
parser.add_argument(
    "--overwrite",
    help="Overwrite existing openwakeword features when the --augment_clips flag is used",
    type=str,
    default="False",
)
parser.add_argument(
    "--train_model",
    help="Execute the model training process",
    type=str,
    default="True",
)

# ✅ 忽略 Jupyter Notebook 传入的 --f 参数
args, unknown = parser.parse_known_args()

if unknown:
    print(f"Ignoring unknown arguments: {unknown}")
config = yaml.load(open(args.training_config, 'r').read(), yaml.Loader)

# imports Piper for synthetic sample generation
sys.path.insert(0, os.path.abspath(config["piper_sample_generator_path"]))


# Define output locations
config["output_dir"] = os.path.abspath(config["output_dir"])
if not os.path.exists(config["output_dir"]):
    os.makedirs(config["output_dir"], exist_ok=True)
if not os.path.exists(os.path.join(config["output_dir"], config["model_name"])):
    os.makedirs(os.path.join(config["output_dir"], config["model_name"]), exist_ok=True)
shutil.copytree(os.path.join(config["datasets"],config["target_phrase"][0],"positive_train"), os.path.join(config["output_dir"],config["model_name"],"positive_train"))
shutil.copytree(os.path.join(config["datasets"],config["target_phrase"][0],"positive_test"), os.path.join(config["output_dir"],config["model_name"],"positive_test"))
shutil.copytree(os.path.join(config["datasets"],config["negative_data"],"negative_train"), os.path.join(config["output_dir"],config["model_name"],"negative_train"))
shutil.copytree(os.path.join(config["datasets"],config["negative_data"],"negative_test"), os.path.join(config["output_dir"],config["model_name"],"negative_test"))

positive_train_output_dir = os.path.join(config["output_dir"], config["model_name"], "positive_train")
positive_test_output_dir = os.path.join(config["output_dir"], config["model_name"], "positive_test")
negative_train_output_dir = os.path.join(config["output_dir"], config["model_name"], "negative_train")
negative_test_output_dir = os.path.join(config["output_dir"], config["model_name"], "negative_test")
feature_save_dir = os.path.join(config["output_dir"], config["model_name"])

# Get paths for impulse response and background audio files
rir_paths = [i.path for j in config["rir_paths"] for i in os.scandir(j)]
background_paths = []
if len(config["background_paths_duplication_rate"]) != len(config["background_paths"]):
    config["background_paths_duplication_rate"] = [1]*len(config["background_paths"])
for background_path, duplication_rate in zip(config["background_paths"], config["background_paths_duplication_rate"]):
    background_paths.extend([i.path for i in os.scandir(background_path)]*duplication_rate)


# Generate positive clips for training
logging.info("#"*50 + "\nGenerating positive clips for training\n" + "#"*50)
if not os.path.exists(positive_train_output_dir):
    os.makedirs(positive_train_output_dir, exist_ok=True)
n_current_samples = len(os.listdir(positive_train_output_dir))
logger.info(f"Current number of positive training samples: {n_current_samples}")

# Generate positive clips for testing
logging.info("#"*50 + "\nGenerating positive clips for testing\n" + "#"*50)
if not os.path.exists(positive_test_output_dir):
    os.makedirs(positive_test_output_dir, exist_ok=True)
n_current_samples = len(os.listdir(positive_test_output_dir))
logger.info(f"Current number of positive testing samples: {n_current_samples}")


# Generate adversarial negative clips for training
logging.info("#"*50 + "\nGenerating negative clips for training\n" + "#"*50)
if not os.path.exists(negative_train_output_dir):
    os.makedirs(negative_train_output_dir, exist_ok=True)
n_current_samples = len(os.listdir(negative_train_output_dir))
logger.info(f"Current number of negative training samples: {n_current_samples}")


# Generate adversarial negative clips for testing
logging.info("#"*50 + "\nGenerating negative clips for testing\n" + "#"*50)
if not os.path.exists(negative_test_output_dir):
    os.makedirs(negative_test_output_dir, exist_ok=True)
n_current_samples = len(os.listdir(negative_test_output_dir))
logger.info(f"Current number of negative testing samples: {n_current_samples}")


# Set the total length of the training clips based on the ~median generated clip duration, rounding to the nearest 1000 samples
# and setting to 32000 when the median + 750 ms is close to that, as it's a good default value
n = 50  # sample size
positive_clips = [str(i) for i in Path(positive_test_output_dir).glob("*.wav")]
duration_in_samples = []
for i in range(n):
    sr, dat = scipy.io.wavfile.read(positive_clips[np.random.randint(0, len(positive_clips))])
    duration_in_samples.append(len(dat))

config["total_length"] = int(round(np.median(duration_in_samples)/1000)*1000) + 12000  # add 750 ms to clip duration as buffer
if config["total_length"] < 32000:
    config["total_length"] = 32000  # set a minimum of 32000 samples (2 seconds)
elif abs(config["total_length"] - 32000) <= 4000:
    config["total_length"] = 32000


/home/ubuntu/miniconda3/envs/train_openwake/lib/python3.10/site-packages/pronouncing/__init__.py:3: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import resource_stream
/home/ubuntu/miniconda3/envs/train_openwake/lib/python3.10/site-packages/torch_audiomentations/utils/io.py:27: UserWarning: torchaudio._backend.set_audio_backend has been deprecated. With dispatcher enabled, this function is no-op. You can remove the function call.
  torchaudio.set_audio_backend("soundfile")


Ignoring unknown arguments: ['--f=/run/user/1000/jupyter/runtime/kernel-v34107afa2f45d99960038d4fdcdb7bcda88322be0.json']


2025-10-28 06:38:02.969 | INFO     | __main__:<module>:120 - Current number of positive training samples: 14992
2025-10-28 06:38:02.971 | INFO     | __main__:<module>:127 - Current number of positive testing samples: 1660
2025-10-28 06:38:02.981 | INFO     | __main__:<module>:135 - Current number of negative training samples: 18320
2025-10-28 06:38:02.982 | INFO     | __main__:<module>:143 - Current number of negative testing samples: 1043
/tmp/ipykernel_3271282/152458878.py:152: WavFileWarning: Reached EOF prematurely; finished at 110483 bytes, expected 4294967303 bytes from header.
  sr, dat = scipy.io.wavfile.read(positive_clips[np.random.randint(0, len(positive_clips))])
/tmp/ipykernel_3271282/152458878.py:152: WavFileWarning: Reached EOF prematurely; finished at 83735 bytes, expected 4294967303 bytes from header.
  sr, dat = scipy.io.wavfile.read(positive_clips[np.random.randint(0, len(positive_clips))])
/tmp/ipykernel_3271282/152458878.py:152: WavFileWarning: Reached EOF prematur

In [4]:

# Do Data Augmentation

if not os.path.exists(os.path.join(feature_save_dir, "positive_features_train.npy")) or args.overwrite is True:
    positive_clips_train = [str(i) for i in Path(positive_train_output_dir).glob("*.wav")]*config["augmentation_rounds"]
    positive_clips_train_generator = augment_clips(positive_clips_train, total_length=config["total_length"],
                                                    batch_size=config["augmentation_batch_size"],
                                                    background_clip_paths=background_paths,
                                                    RIR_paths=rir_paths)

    positive_clips_test = [str(i) for i in Path(positive_test_output_dir).glob("*.wav")]*config["augmentation_rounds"]
    positive_clips_test_generator = augment_clips(positive_clips_test, total_length=config["total_length"],
                                                    batch_size=config["augmentation_batch_size"],
                                                    background_clip_paths=background_paths,
                                                    RIR_paths=rir_paths)

    negative_clips_train = [str(i) for i in Path(negative_train_output_dir).glob("*.wav")]*config["augmentation_rounds"]
    negative_clips_train_generator = augment_clips(negative_clips_train, total_length=config["total_length"],
                                                    batch_size=config["augmentation_batch_size"],
                                                    background_clip_paths=background_paths,
                                                    RIR_paths=rir_paths)

    negative_clips_test = [str(i) for i in Path(negative_test_output_dir).glob("*.wav")]*config["augmentation_rounds"]
    negative_clips_test_generator = augment_clips(negative_clips_test, total_length=config["total_length"],
                                                    batch_size=config["augmentation_batch_size"],
                                                    background_clip_paths=background_paths,
                                                    RIR_paths=rir_paths)

    # Compute features and save to disk via memmapped arrays
    logging.info("#"*50 + "\nComputing openwakeword features for generated samples\n" + "#"*50)
    n_cpus = os.cpu_count()
    if n_cpus is None:
        n_cpus = 1
    else:
        n_cpus = n_cpus//2
    compute_features_from_generator(positive_clips_train_generator, n_total=len(os.listdir(positive_train_output_dir)),
                                    clip_duration=config["total_length"],
                                    output_file=os.path.join(feature_save_dir, "positive_features_train.npy"),
                                    device="gpu" if torch.cuda.is_available() else "cpu",
                                    ncpu=n_cpus if not torch.cuda.is_available() else 1)
    compute_features_from_generator(negative_clips_train_generator, n_total=len(os.listdir(negative_train_output_dir)),
                                    clip_duration=config["total_length"],
                                    output_file=os.path.join(feature_save_dir, "negative_features_train.npy"),
                                    device="gpu" if torch.cuda.is_available() else "cpu",
                                    ncpu=n_cpus if not torch.cuda.is_available() else 1)

    compute_features_from_generator(positive_clips_test_generator, n_total=len(os.listdir(positive_test_output_dir)),
                                    clip_duration=config["total_length"],
                                    output_file=os.path.join(feature_save_dir, "positive_features_test.npy"),
                                    device="gpu" if torch.cuda.is_available() else "cpu",
                                    ncpu=n_cpus if not torch.cuda.is_available() else 1)

    compute_features_from_generator(negative_clips_test_generator, n_total=len(os.listdir(negative_test_output_dir)),
                                    clip_duration=config["total_length"],
                                    output_file=os.path.join(feature_save_dir, "negative_features_test.npy"),
                                    device="gpu" if torch.cuda.is_available() else "cpu",
                                    ncpu=n_cpus if not torch.cuda.is_available() else 1)
else:
    logging.warning("Openwakeword features already exist, skipping data augmentation and feature generation")


/home/ubuntu/miniconda3/envs/train_openwake/lib/python3.10/site-packages/onnxruntime/capi/onnxruntime_inference_collection.py:115: UserWarning: Specified provider 'CUDAExecutionProvider' is not in available provider names.Available providers: 'AzureExecutionProvider, CPUExecutionProvider'
  warnings.warn(
/home/ubuntu/miniconda3/envs/train_openwake/lib/python3.10/site-packages/torch_audiomentations/core/transforms_interface.py:77: FutureWarning: Transforms now expect an `output_type` argument that currently defaults to 'tensor', will default to 'dict' in v0.12, and will be removed in v0.13. Make sure to update your code to something like:
  >>> augment = PitchShift(..., output_type='dict')
  >>> augmented_samples = augment(samples).samples
  warnings.warn(
/home/ubuntu/miniconda3/envs/train_openwake/lib/python3.10/site-packages/torch_audiomentations/core/transforms_interface.py:77: FutureWarning: Transforms now expect an `output_type` argument that currently defaults to 'tensor', will 

In [5]:

# Create openwakeword model
from openwakeword.utils import AudioFeatures
F = AudioFeatures(device='cpu')
input_shape = np.load(os.path.join(feature_save_dir, "positive_features_test.npy")).shape[1:]

oww = Model(n_classes=1, input_shape=input_shape, model_type=config["model_type"],
            layer_dim=config["layer_size"], seconds_per_example=1280*input_shape[0]/16000)

# Create data transform function for batch generation to handle differ clip lengths (todo: write tests for this)
def f(x, n=input_shape[0]):
    """Simple transformation function to ensure negative data is the appropriate shape for the model size"""
    if n > x.shape[1] or n < x.shape[1]:
        x = np.vstack(x)
        new_batch = np.array([x[i:i+n, :] for i in range(0, x.shape[0]-n, n)])
    else:
        return x
    return new_batch

# Create label transforms as needed for model (currently only supports binary classification models)
data_transforms = {key: f for key in config["feature_data_files"].keys()}
label_transforms = {}
for key in ["positive"] + list(config["feature_data_files"].keys()) + ["adversarial_negative"]:
    if key == "positive":
        label_transforms[key] = lambda x: [1 for i in x]
    else:
        label_transforms[key] = lambda x: [0 for i in x]

# Add generated positive and adversarial negative clips to the feature data files dictionary
config["feature_data_files"]['positive'] = os.path.join(feature_save_dir, "positive_features_train.npy")
config["feature_data_files"]['adversarial_negative'] = os.path.join(feature_save_dir, "negative_features_train.npy")

# Make PyTorch data loaders for training and validation data
batch_generator = mmap_batch_generator(
    config["feature_data_files"],
    n_per_class=config["batch_n_per_class"],
    data_transform_funcs=data_transforms,
    label_transform_funcs=label_transforms
)

class IterDataset(torch.utils.data.IterableDataset):
    def __init__(self, generator):
        self.generator = generator

    def __iter__(self):
        return self.generator

n_cpus = os.cpu_count()
if n_cpus is None:
    n_cpus = 1
else:
    n_cpus = n_cpus//2
X_train = torch.utils.data.DataLoader(IterDataset(batch_generator),
                                        batch_size=None, num_workers=n_cpus, prefetch_factor=16)

X_val_fp = np.load(config["false_positive_validation_data_path"])
X_val_fp = np.array([X_val_fp[i:i+input_shape[0]] for i in range(0, X_val_fp.shape[0]-input_shape[0], 1)])  # reshape to match model
X_val_fp_labels = np.zeros(X_val_fp.shape[0]).astype(np.float32)
X_val_fp = torch.utils.data.DataLoader(
    torch.utils.data.TensorDataset(torch.from_numpy(X_val_fp), torch.from_numpy(X_val_fp_labels)),
    batch_size=len(X_val_fp_labels)
)

X_val_pos = np.load(os.path.join(feature_save_dir, "positive_features_test.npy"))
X_val_neg = np.load(os.path.join(feature_save_dir, "negative_features_test.npy"))
labels = np.hstack((np.ones(X_val_pos.shape[0]), np.zeros(X_val_neg.shape[0]))).astype(np.float32)

X_val = torch.utils.data.DataLoader(
    torch.utils.data.TensorDataset(
        torch.from_numpy(np.vstack((X_val_pos, X_val_neg))),
        torch.from_numpy(labels)
        ),
    batch_size=len(labels)
)

# Run auto training
best_model = oww.auto_train(
    X_train=X_train,
    X_val=X_val,
    false_positive_val_data=X_val_fp,
    steps=config["steps"],
    max_negative_weight=config["max_negative_weight"],
    target_fp_per_hour=config["target_false_positives_per_hour"],
)

# Export the trained model to onnx
oww.export_model(model=best_model, model_name=config["model_name"], output_dir=os.path.join(config["output_dir"], config["model_name"]))

# Convert the model from onnx to tflite format
convert_onnx_to_tflite(os.path.join(config["output_dir"],config["model_name"], config["model_name"] + ".onnx"),
                        os.path.join(config["output_dir"],config["model_name"], config["model_name"] + ".tflite"))


Training: 100%|█████████▉| 19999/20000 [03:49<00:00, 87.02it/s] 
Training: 100%|█████████▉| 1999/2000.0 [01:56<00:00, 17.15it/s]
Training: 100%|█████████▉| 1999/2000.0 [01:55<00:00, 17.29it/s]
2025-10-28 07:01:35.445649: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-10-28 07:01:37.759028: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


Estimated count of arithmetic ops: 0.101 M  ops, equivalently 0.050 M  MACs


2025-10-28 07:01:37.984857: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:357] Ignored output_format.
2025-10-28 07:01:37.984882: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:360] Ignored drop_control_dependency.
2025-10-28 07:01:37.985527: I tensorflow/cc/saved_model/reader.cc:43] Reading SavedModel from: /tmp/tmprdqwcpru/tf_model
2025-10-28 07:01:37.985969: I tensorflow/cc/saved_model/reader.cc:78] Reading meta graph with tags { serve }
2025-10-28 07:01:37.985982: I tensorflow/cc/saved_model/reader.cc:119] Reading SavedModel debug info (if present) from: /tmp/tmprdqwcpru/tf_model
2025-10-28 07:01:37.987710: I tensorflow/cc/saved_model/loader.cc:228] Restoring SavedModel bundle.
2025-10-28 07:01:38.001740: I tensorflow/cc/saved_model/loader.cc:212] Running initialization op on SavedModel bundle at path: /tmp/tmprdqwcpru/tf_model
2025-10-28 07:01:38.008157: I tensorflow/cc/saved_model/loader.cc:301] SavedModel load for tags { serve }; Status